In [1]:
import numpy as np
import gymnasium as gym

In [6]:
env = gym.make("FrozenLake-v1")
print(env.observation_space)
print(env.action_space)
print(env.spec.reward_threshold)
print(env.spec.max_episode_steps)

Discrete(16)
Discrete(4)
0.7
100


In [16]:
P = env.env.env.env.P
P

{0: {0: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 4, 0.0, False)],
  1: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 4, 0.0, False),
   (0.3333333333333333, 1, 0.0, False)],
  2: [(0.3333333333333333, 4, 0.0, False),
   (0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)],
  3: [(0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)]},
 1: {0: [(0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 5, 0.0, True)],
  1: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 5, 0.0, True),
   (0.3333333333333333, 2, 0.0, False)],
  2: [(0.3333333333333333, 5, 0.0, True),
   (0.3333333333333333, 2, 0.0, False),
   (0.3333333333333333, 1, 0.0, False)],
  3: [(0.3333333333333333, 2, 0.0, False),
   (0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)]},
 2:

In [18]:
P[14]

{0: [(0.3333333333333333, 10, 0.0, False),
  (0.3333333333333333, 13, 0.0, False),
  (0.3333333333333333, 14, 0.0, False)],
 1: [(0.3333333333333333, 13, 0.0, False),
  (0.3333333333333333, 14, 0.0, False),
  (0.3333333333333333, 15, 1.0, True)],
 2: [(0.3333333333333333, 14, 0.0, False),
  (0.3333333333333333, 15, 1.0, True),
  (0.3333333333333333, 10, 0.0, False)],
 3: [(0.3333333333333333, 15, 1.0, True),
  (0.3333333333333333, 10, 0.0, False),
  (0.3333333333333333, 13, 0.0, False)]}

In [20]:
env # Location alone cannot fully determine the state, rather location + step.

<TimeLimit<OrderEnforcing<PassiveEnvChecker<FrozenLakeEnv<FrozenLake-v1>>>>>

In [21]:
def play_policy(env, policy, render=False):
    episode_reward = 0
    observation, _ = env.reset()
    while True:
        if render: env.render()
        action = np.random.choice(env.action_space.n, p=policy[observation])
        observation, reward, terminated, truncated, _ = env.step(action)
        episode_reward += reward
        if terminated or truncated:
            break
    return episode_reward

In [22]:
# Create a random policy that selects each action via a uniform distribution.
random_policy = np.ones((env.observation_space.n, env.action_space.n)) / env.action_space.n

episode_rewards = [play_policy(env, random_policy) for _ in range(100)]
np.mean(episode_rewards), np.std(episode_rewards)

(np.float64(0.01), np.float64(0.09949874371066199))

## Book Implementation of Policy Evaluation

In [23]:
def v2q(env, P, v, state=None, gamma=1):
    # Calculate action value from state value
    if state is not None:
        q = np.zeros(env.action_space.n)
        for action in range(env.action_space.n):
            for prob, next_state, reward, terminated in P[state][action]:
                q[action] += prob * (reward + gamma * v[next_state] * (1 - terminated))
    else:
        q = np.zeros((env.observation_space.n, env.action_space.n))
        for state in range(env.observation_space.n):
            q[state] = v2q(env, P, v, state, gamma)
    return q

In [40]:
rewards = set()

n_states = env.observation_space.n
n_actions = env.action_space.n
n_rewards = 2

P_tensor = np.zeros((n_states, n_rewards, n_states, n_actions))
terminals = np.zeros(n_states).astype(bool)
for state in P:
    s_d = P[state]
    for action in s_d:
        sa_d = s_d[action]
        for prob, next_state, reward, terminated in sa_d:
            P_tensor[next_state, int(reward), state, action] += prob
            terminals[next_state] = terminals[next_state] or terminated
P_tensor.shape

(16, 2, 16, 4)

In [47]:
terminals = terminals.astype(int)
rewards = np.array([0,1])
terminals

array([0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1])

In [48]:
np.sum(P_tensor, axis=(0,1))

array([[1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.]])

In [ ]:
def v2q_vectorised(P_tensor, rewards, v, terminals, gamma=1):
    # Calculate action value from state value
    r_sa = np.sum(rewards[np.newaxis, :, np.newaxis, np.newaxis] * P_tensor, axis=(0,1)) # (S, A)
    q = r_sa + gamma * np.sum(np.sum(P_tensor, axis=1) * 
                              v[:, np.newaxis, np.newaxis] *
                              (1 - terminals[:, np.newaxis, np.newaxis]), axis=0)
    return q

In [58]:
v_test = np.random.rand(n_states)
v2q_vectorised(P_tensor, rewards, v_test, terminals)

array([[0.28239766, 0.24514864, 0.24514864, 0.09391937],
       [0.05019657, 0.05260404, 0.01535501, 0.05907781],
       [0.059674  , 0.09790518, 0.10031264, 0.06246743],
       [0.05599365, 0.05599365, 0.09422483, 0.10310607],
       [0.25928236, 0.21555956, 0.06433029, 0.23867486],
       [0.        , 0.        , 0.        , 0.        ],
       [0.23759676, 0.22871553, 0.23759676, 0.00888124],
       [0.        , 0.        , 0.        , 0.        ],
       [0.21555956, 0.11816306, 0.29250764, 0.31311513],
       [0.0232352 , 0.25195072, 0.23134323, 0.24932302],
       [0.17227369, 0.1279547 , 0.07471812, 0.14187456],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.10018328, 0.03302684, 0.13058241, 0.1279547 ],
       [0.26174236, 0.36636017, 0.59244799, 0.56467656],
       [0.        , 0.        , 0.        , 0.        ]])

In [59]:
v2q(env, P, v_test)

array([[0.28239766, 0.24514864, 0.24514864, 0.09391937],
       [0.05019657, 0.05260404, 0.01535501, 0.05907781],
       [0.059674  , 0.09790518, 0.10031264, 0.06246743],
       [0.05599365, 0.05599365, 0.09422483, 0.10310607],
       [0.25928236, 0.21555956, 0.06433029, 0.23867486],
       [0.        , 0.        , 0.        , 0.        ],
       [0.23759676, 0.22871553, 0.23759676, 0.00888124],
       [0.        , 0.        , 0.        , 0.        ],
       [0.21555956, 0.11816306, 0.29250764, 0.31311513],
       [0.0232352 , 0.25195072, 0.23134323, 0.24932302],
       [0.17227369, 0.1279547 , 0.07471812, 0.14187456],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.10018328, 0.03302684, 0.13058241, 0.1279547 ],
       [0.26174236, 0.36636017, 0.59244799, 0.56467656],
       [0.        , 0.        , 0.        , 0.        ]])